# Assignment-07&08

##  1. Using the new data to test different models
 + KNN, Logistic Regression, SVM, Bayes, Random Forest
 + To Classify if a new a by Xinhua

### 任务描述
报社等相关的机构，往往会遇到一个问题，就是别人家的机构使用自己的文章但是并没有标明来源。 在本次任务中，我们将解决新华社的文章被抄袭引用的问题。

给定的数据集合中，存在一些新闻预料，该预料是来自新华社，但是其来源并不是新华社，请设计技巧学习模型解决该问题。

### Step1: 数据分析
请在课程的GitHub中下载数据集，然后使用pandas进行读取。

In [1]:
filepath = "/home/aistudio/data/data10734/sqlResult_1558435.csv"

In [2]:
import pandas as pd

In [3]:
news_data = pd.read_csv(filepath, encoding='gb18030')

In [4]:
news_data = news_data.fillna('')

### Step2: 数据预处理
将pandas中的数据，依据是否是新华社的文章，请改变成新的数据dataframe: <content, y>, 其中，content是文章内容，y是0或者1. 你可能要使用到pandas的dataframe操作。https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.apply.html

+ 2.1 先将语料中的文章进行jieba分词

In [5]:
!pip install jieba

Looking in indexes: https://pypi.mirrors.ustc.edu.cn/simple/
    100% |████████████████████████████████| 7.3MB 3.8MB/s ta 0:00:011
  Running setup.py bdist_wheel for jieba ... done
  Stored in directory: /home/aistudio/.cache/pip/wheels/46/2b/ce/f70aa4307a088454e032e45b63e6447d9ee70bcca85091704e
Successfully built jieba


In [6]:
import re
import jieba

In [7]:
def token(string):
    str_list = re.findall(r'[\d|\w]+', string)
    pre_str = ''.join(str_list)
    # print (temp_str)
    if len(pre_str) == 0: pass
    else: return ' '.join(jieba.cut(re.sub('n', '', pre_str)))

In [8]:
news_data['content'] = news_data['content'].fillna('').apply(token)

Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
Loading model cost 1.134 seconds.
Prefix dict has been built succesfully.


+ 2.2 随机选一篇新闻查看分词后的效果

In [9]:
news_data['content'][5]

'受到 A股 被 纳入 MSCI 指数 的 利好 消息 刺激 A股 市场 从 周三 开始 再度 上演 龙马 行情 周四 上午 金融股 和 白马股 表现 喜人 但是 尾盘 跳水 之后 仅 金融 板块 仍 维系 红盘 状态 分析 人士 认为 金融股 受益 于 MSCI 纳入 A股 和 低 估值 而 重获 资金 青睐 但是 存量 资金 博弈 格局 下 风格 交替 的 震荡 格局 料 延续 流动性 改善 经济 悲观 预期 修正 等 有助于 支撑 板块 继而 大盘 指数 逐步 向 好 一九 再现 周四 A股 市场 未能 延续 周三 的 上行 态势 两市 成交 小幅 放量 29 个 中信 一级 行业 中 收盘 仅 银行 和 非银行 金融 两个 行业 指数 收红 分别 上涨 180 和 020 从 二级 行业 来看 股份制 与 城商行 的 涨幅 最高 达到 222 国有银行 上涨 082 信托 及其 他 上涨 064 保险 板块 上涨 034 证券 板块 上涨 006 银行 板块 25 只 成分股 中共 有 21 只 收红 其中 招商银行 涨幅 最大 上涨 666 贵阳 银行 上涨 365 上海银行 华夏银行 浦发银行 和 兴业银行 的 涨幅 均 超过 150 非银行 金融 板块 44 只 成分股 中共 17 只 个股 上涨 其中 安信 信托 中国 太保 涨幅 居前 两名 分别 上涨 457 和 304 西水股份 华安 证券 中国 人寿 和 新华 保险 的 涨幅 也 均 超过 2 相对而言 券商 股多 小幅 下跌 近期 对 A股 市场 消息面 影响 最大 的 就是 MSCI 宣布 从 2018 年 6 月 开始 将 A股 纳入 MSCI 新兴 市场 指数 而 其中 金融股 是 占 比 最大 的 一个 群体 国金 证券 李立峰 团队 指出 最新 方案 中 包含 的 222 只 成分股 中 剔除 了 中等 市值 非 互联互通 可 交易 的 股票 以及 有 停牌 限制 的 标的 由于 纳入 了 很多 大 市值 A H股 A股 在 MSCIEM 中 的 权重 由 05 上升 到 了 073 其中 金融 板块 占 比 最高 达到 4011 泛 消费 次之 占 比 为 2426 两个 板块 涵盖 了 大部分 权重股 动态 来看 由于 加入 了 很多 是 指 占 比高 的 金融公司 金融 板块

+ 2.3 将总样本（新闻语料）分为正样本（新华网新闻）和负样本（非新华网新闻），并将空的新闻语料剔除

In [10]:
news_data_pos = news_data[news_data['source'].str.contains('新华')]
news_data_pos = news_data_pos.fillna('')

news_data_neg = news_data[news_data['source'].str.contains('新华') == 0]
news_data_neg = news_data_neg.fillna('')

In [11]:
news_data_pos = news_data_pos[news_data_pos["content"] != '']
print (len(news_data_pos))
news_data_neg = news_data_neg[news_data_neg["content"] != '']
print (len(news_data_neg))

78666
8197


从上面的结果中可以看出，正样本的数量为78666个，负样本的数量为8197个，为了使正负样本均衡，我们从正样本中随机选出8197个

In [12]:
news_data_pos = news_data_pos.sample(n=len(news_data_neg))
print (len(news_data_pos))
print (len(news_data_neg))

8197
8197


现在正负样本的数量相同，已经很均衡了 :)

In [13]:
news_data_sampled = news_data_pos.append(news_data_neg)
print (len(news_data_sampled))

16394


+ 2.4 下面我们要为预处理好的文本内容打上标签了，新华网对应的标签为1，非新华网对应的标签为0

In [14]:
import numpy as np

In [15]:
news_data_sampled['is_xinhua'] = np.where(news_data_sampled['source'].str.contains('新华'),1,0)

+ 2.5 先选出一些数据作为test_data（占总样本的比例为10%）

In [16]:
test_data = news_data_sampled.sample(n=int(16394*0.1))
print (len(test_data))

1639


从news_data_sampled中去除掉test_data

In [17]:
news_data_sampled = news_data_sampled.append(test_data)

In [18]:
len(news_data_sampled)

18033

In [19]:
columns = news_data_sampled.columns.tolist()

In [20]:
columns

['id', 'author', 'source', 'content', 'feature', 'title', 'url', 'is_xinhua']

In [21]:
news_data_sampled = news_data_sampled.drop_duplicates(subset=columns,keep=False)

In [22]:
len(news_data_sampled)

14755

经过上述步骤，我们成功将test_data从news_data_sampled中剔除

In [23]:
x_inputs = news_data_sampled.loc[:,"content"].values

In [24]:
x_inputs_test = test_data.loc[:,"content"].values

In [25]:
x_inputs_total = np.append(x_inputs,x_inputs_test)

In [26]:
len(x_inputs_total)

16394

In [27]:
y_inputs = news_data_sampled['is_xinhua'].values

In [28]:
y_inputs_test = test_data['is_xinhua'].values

In [29]:
print(len(news_data_sampled[news_data_sampled['source'] != '新华社']))
print(len(news_data_sampled[news_data_sampled['source'] == '新华社']))

7372
7383


In [30]:
len(y_inputs)

14755

### Step3: 使用tfidf进行文本向量化

参考 https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html 对文本进行向量化

In [31]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [32]:
vectorized = TfidfVectorizer(max_features=10000)

In [33]:
vec_feature = vectorized.fit_transform(x_inputs_total)

In [34]:
vec_feature.shape

(16394, 10000)

In [35]:
np_vecs_total = vec_feature.toarray()

In [36]:
np_vecs = np_vecs_total[0:14755,:]
np_vecs_test = np_vecs_total[14755:,:]

### Step4: 参考scikit-learning的方法，构建你的第一个机器学习模型
+ 按照课程讲解的内容，把数据集分割为 traning_data, validation_data, test_data. https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html
+ 参照scikit learning的示例，从构建你的第一个KNN机器学习模型。

In [37]:
from sklearn.model_selection import train_test_split

x_train, x_validation, y_train, y_validation = train_test_split(
    np_vecs , y_inputs, train_size=0.8, test_size=0.2, random_state=0)

In [38]:
import timeit

In [39]:
def clock(func):
    def clocked(*args, **kwargs):
        t0 = timeit.default_timer()
        result = func(*args, **kwargs)
        elapsed = timeit.default_timer() - t0
        print(elapsed,'s')
        return result
    return clocked

In [40]:
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import roc_auc_score
# from sklearn.metrics import confusion_matrix

def get_performance(clf, x_, y_):
    y_hat = clf.predict(x_)
    print('f1_score is: {}'.format(f1_score(y_, y_hat)))
    print('accuracy is: {}'.format(accuracy_score(y_, y_hat)))
    print('percision is: {}'.format(precision_score(y_, y_hat)))
    print('recall is: {}'.format(recall_score(y_, y_hat)))
    print('roc_auc is: {}'.format(roc_auc_score(y_, y_hat)))
    #print('confusion matrix: \n{}'.format(confusion_matrix(y_, y_hat, labels=[0, 1])))

**KNN机器学习模型：**

In [41]:
from sklearn.neighbors import KNeighborsClassifier
@clock
def KNN(x_train, y_train):
    knn = KNeighborsClassifier(n_neighbors=3, n_jobs=-1)
    knn.fit(x_train, y_train)
    print("mean accuracy on the given test data and labels: ", knn.score(x_train, y_train))
    return knn

In [42]:
knn_model = KNN(x_train, y_train)

mean accuracy on the given test data and labels:  0.8537783802100982
341.2509812116623 s


### Step5: 在traning_data, validation_data, test_data 上观察其相关metric: recall, precision, f1等， 并解释其含义. 

+ 5.1 在traning_data上观察其相关metric

In [43]:
get_performance(knn_model,x_train, y_train)

f1_score is: 0.8316754437292764
accuracy is: 0.8537783802100982
percision is: 0.9886389983769998
recall is: 0.7177242888402626
roc_auc is: 0.8546833963389442


+ 5.2 在validation_data上观察其相关metric

In [44]:
get_performance(knn_model,x_validation, y_validation)

f1_score is: 0.6845342706502635
accuracy is: 0.756692646560488
percision is: 0.9617283950617284
recall is: 0.5313778990450204
roc_auc is: 0.7552512390847999


从上面的结果中可以看出，训练出来的KNN模型在训练集上表现还算好，但是在验证集上的性能评估指标是真的差劲，综上可以初步猜测模型产生了过拟合。下面依次解释各性能评估指标的具体含义：
+ accuracy 准确率：模型正确分类的数量占总样本数量的百分比；
+ percision 精确率：在预测为正类的数据中，有多少预测正确；
+ recall 召回率：在所有正类的数据中，有多少预测正确；
+ f1_score F1值：percision和recall的调和均值；
+ roc_auc：AUC的值为ROC曲线和坐标轴所围成的面积，越接近1表示模型的性能越好。

### Step6: 使用 test_data 对其进行新数据的判断, 你有没

这部分我理解的意思是：在测试数据上观察模型表现的好坏

In [45]:
get_performance(knn_model, np_vecs_test, y_inputs_test)

f1_score is: 0.6882591093117408
accuracy is: 0.7651006711409396
percision is: 0.9550561797752809
recall is: 0.5379746835443038
roc_auc is: 0.7572087787568398


从上面的性能指标中可以看出模型在测试数据上的表现很差

### Step7: 调整不同的参数，观察变化

在这一步中，我们将“n_neighbors”参数从3调整为2

In [46]:
from sklearn.neighbors import KNeighborsClassifier
@clock
def KNN(x_train, y_train):
    knn = KNeighborsClassifier(n_neighbors=2, n_jobs=-1)
    knn.fit(x_train, y_train)
    print("mean accuracy on the given test data and labels: ", knn.score(x_train, y_train))
    return knn

In [47]:
knn_model_1 = KNN(x_train, y_train)

mean accuracy on the given test data and labels:  0.8359030837004405
281.647005751729 s


+ 7.1 在traning_data上观察其相关metric

In [48]:
get_performance(knn_model_1,x_train, y_train)

f1_score is: 0.8052287581699347
accuracy is: 0.8359030837004405
percision is: 1.0
recall is: 0.6739606126914661
roc_auc is: 0.8369803063457331


+ 7.2 在validation_data上观察其相关metric

In [49]:
get_performance(knn_model_1, x_validation, y_validation)

f1_score is: 0.667866786678668
accuracy is: 0.7499152829549305
percision is: 0.9814814814814815
recall is: 0.5061391541609823
roc_auc is: 0.7483557723666864


+ 7.3 在test_data上观察其相关metric

In [50]:
get_performance(knn_model_1, np_vecs_test, y_inputs_test)

f1_score is: 0.6727272727272727
accuracy is: 0.7583892617449665
percision is: 0.969047619047619
recall is: 0.5151898734177215
roc_auc is: 0.7499388707489079


从上面的结果中可以看出，将“n_neighbors”参数从3调整为2之后模型的性能并没有发生显著的提升，所以接下来使用scikit learning 中的GridSearchCV进行批量调参

In [55]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
@clock
def KNN(x_train, y_train):
    knn = KNeighborsClassifier()
    param_grid = { 'n_neighbors' : [i for i in range(1,6)]}
    clf = GridSearchCV(knn, param_grid, cv = 2, scoring='recall', n_jobs = -1)
    clf.fit(x_train, y_train)
    print('the optimal number of neighbors: ', clf.best_params_)
    print("mean accuracy on the given test data and labels: ", clf.score(x_train, y_train))
    return clf

In [56]:
knn_model_2 = KNN(x_train, y_train)

the optimal number of neighbors:  {'n_neighbors': 1}
mean accuracy on the given test data and labels:  0.999663356337317
1438.9814233165234 s


从上面的结果中可以看出，n_neighbors在区间[1,6]中选择1是最优的，我们接下来用该模型分别在训练、验证和测试集上观察性能指标

In [57]:
get_performance(knn_model_2, x_train, y_train)

f1_score is: 0.9998316498316498
accuracy is: 0.9998305659098611
percision is: 1.0
recall is: 0.999663356337317
roc_auc is: 0.9998316781686585


In [58]:
get_performance(knn_model_2, x_validation, y_validation)

f1_score is: 0.7650058846606513
accuracy is: 0.7970179600135547
percision is: 0.9002770083102493
recall is: 0.665075034106412
roc_auc is: 0.7961738806895696


In [59]:
get_performance(knn_model_2, np_vecs_test, y_inputs_test)

f1_score is: 0.7666422823701536
accuracy is: 0.8053691275167785
percision is: 0.9081455805892548
recall is: 0.6632911392405063
roc_auc is: 0.800432377629676


从上述结果中可以看出，n_neighbors=1时训练出来的模型总体效果要优于前两个模型（n_neighbors=2，n_neighbors=3）

### Step8: 不断改变参数，直到性能达到“某个”点。问：“某个”怎么定义？

+ 我觉得“某个”的定义要根据具体的任务和要实现的效果来综合给出，像本次作业中训练出来的新华网文章判断模型，当性能指标accuracy在测试数据集上能够大于0.9，其它指标相对比较OK时，就可以认为训练出来的模型是一个比较好的模型了（能够基本完成所给定的任务），此时便可以终止调参这一步骤。
+ 对于本次作业来说，当训练出来的模型accuracy>0.9时，其实已经比人类的判断要准确多了（我觉得大部分人类准确度达到70%就已经非常不错了），这时候的模型其实已经基本能够完成新华网文章判别任务了。

### Step9: 找出所有预测为 1， 但是实际为 0 的文章。 作为抄袭的候选者。

In [60]:
y_hat = knn_model_2.predict(np_vecs_test)

In [67]:
index_ = []
for i in range(0, len(y_hat)):
    if y_hat[i] == 1 and y_inputs_test[i] == 0:
        index_.append(i)

In [69]:
len(index_)

53

In [74]:
plagiarized_candidate = x_inputs_test[index_]

In [78]:
plagiarized_candidate[24]

'环球网 综合 报道 据 英国 每日 邮报 6 月 12 日 报道 在 葡萄牙 特 塞拉 岛 一名 中年男子 在 斗牛 现场 因 专心 录制 视频 未 发现 一头 狂暴 的 公牛 朝 自己 奔来 来不及 躲闪 被 顶 伤 目前 该 男子 伤势 尚 不 明确 从 视频 中 可以 看到 这位 名叫 达 玆 德 的 观众 正 忙于 用 平板 电脑 拍摄 斗牛 场面 未 注意 到 向 他 奔 去 的 公牛 公牛 先是 将 他 顶 向 空中 后 用 牛角 一直 砸击 达 玆 德 的 头部 和 胸部 达 玆 德 被 顶 向 空中 后 摔 到 水泥 地上 头部 受伤 直到 另 一位 观众 成功 转移 公牛 的 注意力 达 玆 德才 得以 获救 达 玆 德 松开 手时 伤口 有 血流 出特 塞拉 岛 的 居民 通常 在 每年 的 4 月 至 9 月 下旬 举行 斗牛 赛 今年 斗牛 赛 在 小镇 中心 举行 比赛 时 人们 稍稍 松开 公牛 的 绳子 不断 刺激 公牛 但 仍 由 8 人拉着 绳子 以免 公牛 失控 实习 编译 吴露鑫 审稿 朱盈库'

plagiarized_candidate列表存储的就是从测试数据中找到的抄袭的候选者，一共有53篇文章

### Step10： 总结该过程，什么是数据思维？什么是机器学习思维？

+ 数据思维：能够把需求或者业务问题转化为“数据可分析问题”，即能够找出自变量X和因变量Y。对数据进行合理的预处理，并将数据拟合为合适的“函数”。
+ 机器学习思维：能够将“隐式的”机器学习问题转化为“显式的”机器学习问题，就像本次作业中的抄袭新华网文章判断问题，先训练一个能够准确判断新华网文章的模型，如果模型判断一篇文章是新华网的，但是文章的来源不是新华网，那么就可以认定这篇文章是抄袭新华网的，综上就是一个从“隐式”到“显式”的转化，即为机器学习思维。

### Step11: 利用第8课讲述的新模型，进行操作，感受其中不同的参数、模型对性能的影响。

+ **Logistic Regression模型**

In [79]:
from sklearn.linear_model import LogisticRegression
@clock
def logist(x_train, y_train):
    clf = LogisticRegression(solver='saga').fit(x_train, y_train)
    print ("mean accuracy on the given test data and labels: ", clf.score(x_train, y_train))
    return clf

get_performance(logist(x_train,y_train),np_vecs_test,y_inputs_test)

mean accuracy on the given test data and labels:  0.9662826160623518
16.96536340750754 s
f1_score is: 0.9369608493696086
accuracy is: 0.9420378279438683
percision is: 0.9846582984658299
recall is: 0.8936708860759494
roc_auc is: 0.940357233379553


+ **SVM模型**

In [80]:
from sklearn.svm import LinearSVC
@clock
def SVM(x_train, y_train):
    clf = LinearSVC(dual=False)
    clf.fit(x_train, y_train)
    print("mean accuracy on the given test data and labels: ", clf.score(x_train, y_train))
    return clf

get_performance(SVM(x_train,y_train),np_vecs_test,y_inputs_test)

mean accuracy on the given test data and labels:  0.9982209420535412
1.7159994263201952 s
f1_score is: 0.9620578778135048
accuracy is: 0.9640024405125076
percision is: 0.9777777777777777
recall is: 0.9468354430379747
roc_auc is: 0.963405942955972


+ **Bayes模型**

In [81]:
from sklearn.naive_bayes import GaussianNB
@clock
def bayes(x_train, y_train):
    clf = GaussianNB()
    clf.fit(x_train, y_train)
    print("mean accuracy on the given test data and labels: ", clf.score(x_train, y_train))
    return clf

get_performance(bayes(x_train,y_train),np_vecs_test,y_inputs_test)

mean accuracy on the given test data and labels:  0.9485767536428329
6.862606760114431 s
f1_score is: 0.8543177323665129
accuracy is: 0.8651616839536302
percision is: 0.8913342503438789
recall is: 0.8202531645569621
roc_auc is: 0.8636012583679981


In [82]:
from sklearn.naive_bayes import MultinomialNB
@clock
def bayes_multi(x_train, y_train):
    clf = MultinomialNB()
    clf.fit(x_train, y_train)
    print("mean accuracy on the given test data and labels: ", clf.score(x_train, y_train))
    return clf

get_performance(bayes_multi(x_train,y_train),np_vecs_test,y_inputs_test)

mean accuracy on the given test data and labels:  0.8924940698068451
0.5890356041491032 s
f1_score is: 0.8523489932885906
accuracy is: 0.8657718120805369
percision is: 0.9071428571428571
recall is: 0.8037974683544303
roc_auc is: 0.8636184043774506


In [83]:
from sklearn.naive_bayes import BernoulliNB
@clock
def bayes_bernoulli(x_train, y_train):
    clf = BernoulliNB(binarize=None)
    clf.fit(x_train, y_train)
    print("mean accuracy on the given test data and labels: ", clf.score(x_train, y_train))
    return clf

get_performance(bayes_bernoulli(x_train,y_train),np_vecs_test,y_inputs_test)

mean accuracy on the given test data and labels:  0.8773297187394103
0.3489094488322735 s
f1_score is: 0.8402061855670103
accuracy is: 0.8486882245271506
percision is: 0.8556430446194225
recall is: 0.8253164556962025
roc_auc is: 0.8478761312638844


+ **Decision Tree模型**

In [85]:
from sklearn.tree import DecisionTreeClassifier
@clock
def decision_tree(x_train, y_train):
    clf = DecisionTreeClassifier()
    clf.fit(x_train, y_train)
    print("mean accuracy on the given test data and labels: ", clf.score(x_train, y_train))
    return clf

get_performance(decision_tree(x_train,y_train),np_vecs_test,y_inputs_test)

mean accuracy on the given test data and labels:  0.9998305659098611
179.48635926656425 s
f1_score is: 0.968025078369906
accuracy is: 0.9688834655277608
percision is: 0.9590062111801242
recall is: 0.9772151898734177
roc_auc is: 0.9691729659614436


+ **Random Forest模型**

In [86]:
from sklearn.ensemble import RandomForestClassifier
@clock
def random_forest(x_train, y_train):
    clf = RandomForestClassifier(n_estimators=100,random_state=0)
    clf.fit(x_train, y_train)
    print("mean accuracy on the given test data and labels: ", clf.score(x_train, y_train))
    return clf

get_performance(random_forest(x_train,y_train),np_vecs_test,y_inputs_test)

mean accuracy on the given test data and labels:  0.9998305659098611
88.22977033630013 s
f1_score is: 0.9672025723472669
accuracy is: 0.9688834655277608
percision is: 0.9830065359477124
recall is: 0.9518987341772152
roc_auc is: 0.9682933011286546


**总结：从上面各模型的性能结果中可以看出，SVM、Decision Tree和Random Forest模型在本次作业任务中表现突出，其中Random Forest模型在测试数据集上的性能指标最好，但是SVM模型的训练时间最短且在测试数据集上的性能指标和Random Forest模型相差并不是很大。**

***********************************************************************

# Assignment-08

## 2. Summary the advantages and disadvantages about Linear Regression, Logistic Regression, K-Nearest Neighbors, Support Vector Machine, Bayes, etc.

<font color=DeepPink size=3>由于这部分高老师上课没有细讲，看李航的《统计学习方法》也没有分析各个算法的优缺点，所以在网上看了一些博客，最后还是觉得这篇讲的比较详细：</font>
+ 机器学习算法优缺点对比及选择（汇总篇） - 杜博亚的文章 - 知乎
https://zhuanlan.zhihu.com/p/46831267

**上面这篇文章中有些地方还是不能太理解，等学习一段时间后自己再回过头来思考吧。**